# Z500, MSLP vue 3D nord atlantique - animation temporelle

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2022**

In [ ]:
%matplotlib inline

import os

from tqdm import tqdm

import xarray as xr
import numpy as np

import itertools
import datetime as dt

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
from cartopy import config
from cartopy.mpl.patch import geos_to_path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.pyplot import figure
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dir_data='./data/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

In [ ]:
latS=20
latN=80
lonW=-100
lonE=40

year1='1980'
year2='2020'

In [ ]:
city1="Stykkisholmur"
city2="Lisbon"

city1_xy=[-22.5, 65.7]
city2_xy=[-9.1, 38.7]

# Ouverture et traitement des données

In [ ]:
fz    = xr.open_dataset(dir_data+'era5_z500_natl_daily_1950-2020_1deg5.nc').sel(lat=slice(latS,latN)).sel(time=slice(year1,year2))
fp    = xr.open_dataset(dir_data+'era5_msl_natl_daily_1950-2020_1deg5.nc').sel(lat=slice(latS,latN)).sel(time=slice(year1,year2))

time0  = fp.time.values
lat = fp.lat.values
lon = fp.lon.values

z = fz['z']/9.81
msl = fp['msl']/100

print(z)

In [ ]:
z_city1=z.sel(lat=city1_xy[1], method="nearest").sel(lon=city1_xy[0], method="nearest")
z_city1_anom=z_city1.groupby('time.dayofyear') - z_city1.groupby('time.dayofyear').mean('time')
z_city1_anom_std=z_city1_anom/z_city1.groupby('time.dayofyear').std('time')

z_city2=z.sel(lat=city2_xy[1], method="nearest").sel(lon=city2_xy[0], method="nearest")
z_city2_anom=z_city2.groupby('time.dayofyear') - z_city2.groupby('time.dayofyear').mean('time')
z_city2_anom_std=z_city2_anom/z_city2.groupby('time.dayofyear').std('time')

print(z_city1)
print(z_city1_anom)
print(z_city1_anom_std)

In [ ]:
date1 = input("Date 1 : ")
date2 = input("Date 2 : ")

In [ ]:
zz = z.sel(time=slice(date1,date2))
mslp = msl.sel(time=slice(date1,date2))
zz_city1_anom_std = z_city1_anom_std.sel(time=slice(date1,date2))
zz_city2_anom_std = z_city2_anom_std.sel(time=slice(date1,date2))

print(zz_city1_anom_std.shape)
print(zz.shape)

time  = zz.time.values
# Manage dates
time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]
for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]

# Graphiques

In [ ]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
cor=np.corrcoef(zz_city1_anom_std[:,0], zz_city2_anom_std[:,0])
mean1=np.array(zz_city1_anom_std[:,0].mean())
mean2=np.array(zz_city2_anom_std[:,0].mean())
print(cor)
print(mean1)
print(mean2)

In [ ]:
for i in tqdm(range(len(time))):
    
    #print(date_str[i])
    
    fig=plt.figure(figsize=(15, 5))
    fig.suptitle('Z500 Normalized anomaly - '+date1+'-'+date2, fontsize=16)

    ax = plt.subplot(111)
    ax.set_title(date_str[i], loc='center', fontsize=14)
    ax.plot(time,zz_city1_anom_std[:,0], color='blue')
    ax.plot(time,zz_city2_anom_std[:,0], color='red')

    ax.scatter(time[i],zz_city1_anom_std[i,0], color='blue', linewidth=3, linestyle='-', label=city1)
    ax.scatter(time[i],zz_city2_anom_std[i,0], color='red', linewidth=3, linestyle='-', label=city2)

    plt.ylim(-4, 4)
    plt.axhline(0, linestyle='-', color='black')
    plt.axhline(mean1, linestyle='-', color='blue')
    plt.axhline(mean2, linestyle='-', color='red')
    plt.axvline(time[i], linestyle='-', color='black')
    plt.axvline(dt.datetime(int(date1[0:4]), 12, 1), linestyle='--', color='grey')
    plt.axvline(dt.datetime(int(date2[0:4]), 1, 1), linestyle='--', color='grey')
    plt.axvline(dt.datetime(int(date2[0:4]), 2, 1), linestyle='--', color='grey')
    plt.axvline(dt.datetime(int(date2[0:4]), 3, 1), linestyle='--', color='grey')
    plt.axvline(dt.datetime(int(date2[0:4]), 3, 31), linestyle='--', color='grey')
    
    plt.ylabel('Normalized anomaly')
    plt.xlabel('Time')
    #ax.set_xticks([])
    
    plt.legend(loc='upper right')
    # place a text box in upper left in axes coords
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, 'Correlation : '+str(round(cor[0,1],2)), transform=ax.transAxes,
            fontsize=14, verticalalignment='top', bbox=props)

    figname='./anim/Z500_NAO_evol'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.close()

In [ ]:
gif_filepath = './anim/Z500_NAO_evol'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
levels_z = np.arange(4400,6200,50)
levels_msl = np.arange(920,1065,5)
angle = 310 #270 310

for i in tqdm(range(len(time))):
    
    #print(i)
    
    figure(figsize=(15, 15))
    ax = plt.axes(projection="3d", xlim=[lonW, lonE], ylim=[latS, latN], zlim=[0, 350])
    ax.set_title("Mean Sea Level Pressure - Geopotential height at 500 hPa : "+date_str[i],
                 y=1.0, pad=-20, fontsize=14)
    #plt.title("MSLP and Z500 : "+date_str[i])
    
    # Axes, grid
    plt.box(False)
    ax.set(frame_on=False)  # New
    ax.set_zticklabels([])
    ax.set_zticks([])
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    # Background Map
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature("physical", "coastline", "10m")
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs) for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color="black", zorder=0)
    ax.add_collection3d(lc)
    
    # convert z to np array
    gh = np.array(zz[i,:,:])
    mslpp = np.array(mslp[i,:,:])
    
    # Create a second variable with edited Geopotential Height for better 3d representation
    #gh_3d = gh
    gh_3d = gh/1000
    gh_3d = np.exp(gh_3d)

    # Create meshgrid from lats, lons
    x = lon
    y = lat
    x2 = np.append(0, x.flatten())
    y2 = np.append(0, y.flatten())
    x2, y2 = np.meshgrid(lon, lat)
    
    # Create np array from the gh_3d array
    # flattened to one dimension and delete its first item (for better illustration)
    z2 = np.append(0, gh_3d.flatten())
    z2 = np.delete(z2, 0)
    
    # Plot trisurf data (3d plot of Geopotential Height reduced to the 3d variable)
    surf = ax.plot_trisurf(x2.flatten(), y2.flatten(), z2, cmap='jet',
                           linewidth=0.1, vmin=min(z2), vmax=max(z2), alpha=0.6, antialiased=True)
    
    # Create contour plots on level 0 of z axis for Z
    #cf=ax.contourf(x2, y2, gh, levels_z, zdir="z", offset=0, cmap="jet", alpha=0.8, zorder=10, antialiased=False)
    #ax.contour(x2, y2, gh, levels_z, zdir="z", offset=0, colors="k", linewidths=2, zorder=100)
    cbar = plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=np.min(z), vmax=np.max(z)),
                                              cmap="jet"), shrink=0.5, pad=0.01, orientation="vertical")
    cbar.set_label(label="mgp", fontsize=10)
    
    # Create contour plots on level 0 of z axis for MSLP
    cf=ax.contourf(x2, y2, mslpp, levels_msl, zdir="z", offset=0, cmap="jet",
                   extent='both', alpha=0.6, zorder=10, antialiased=False)
    ax.contour(x2, y2, mslpp, levels_msl, zdir="z", offset=0, colors="k", linewidths=2, zorder=100)
    cbar2 = plt.colorbar(cf, shrink=0.5, pad=0.01, orientation="horizontal")
    cbar2.set_label(label="hPa", fontsize=10)
    
    # Plot cities
    ax.plot([-22.5,-22.5],[65.7,65.7],[0,max(z2)+20],"--",color="grey", alpha=1, linewidth=2)
    ax.text(s="Stykkisholmur", x=-22.5, y=65.7, z=max(z2)+20, zorder=1000)
    ax.plot([-9.1,-9.1],[38.7,38.7],[0,max(z2)+20],"--",color="grey", alpha=1, linewidth=2)
    ax.text(s="Lisbon", x=-9.1, y=38.7, z=max(z2)+20, zorder=1000)
    
    # Set view angle
    ax.view_init(25, angle)
        
    figname='./anim/Z500_MSLP_'+date_str[i]
    plt.savefig(figname+'.png', transparent = False, bbox_inches = 'tight', pad_inches = 0)
    
    plt.close()

In [ ]:
gif_filepath = './anim/Z500_MSLP_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)